In [1]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import sklearn


In [2]:
import keras

Using CNTK backend


In [28]:
from keras import layers, models

## COMP 3132 -  Machine Learning with Python
__Week 4 lecture__

### [github.com/kamrik/ML2](https://github.com/kamrik/ML2)

### [slido.com/COMP3132](http://slido.com/COMP3132)

## Plan for today
 - Training, validation, and test sets
 - Loss functions
 

## Boston housing data

In [8]:
from keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) =  boston_housing.load_data()

train_data.shape

(404, 13)

In [9]:
test_data.shape

(102, 13)

## Rescaling the data

In [38]:
# How to check the spread of the data 
train_data

array([[-0.27224633, -0.48361547, -0.43576161, ...,  1.14850044,
         0.44807713,  0.8252202 ],
       [-0.40342651,  2.99178419, -1.33391162, ..., -1.71818909,
         0.43190599, -1.32920239],
       [ 0.1249402 , -0.48361547,  1.0283258 , ...,  0.78447637,
         0.22061726, -1.30850006],
       ..., 
       [-0.40202987,  0.99079651, -0.7415148 , ..., -0.71712291,
         0.07943894, -0.67776904],
       [-0.17292018, -0.48361547,  1.24588095, ..., -1.71818909,
        -0.98764362,  0.42083466],
       [-0.40422614,  2.04394792, -1.20161456, ..., -1.30866202,
         0.23317118, -1.15392266]])

In [26]:
m = train_data.mean(axis=0)
s = train_data.std(axis=0)

train_data -= m
train_data /= s

test_data -= m
test_data /= s

## Data type

In [40]:
train_data = train_data.astype('float32')
test_data = test_data.astype('float32')
train_targets = train_targets.astype('float32')
test_targets = test_targets.astype('float32')

## Creating the model

## Is this a regression or classification model?

## What metrics are appropriate to evaluate this type of model?

## SKlearn way

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

## Keras way

In [27]:
def create_model():
    # Create a new model
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(13,)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

## We only have about 400 training samples
#### What problems do you anticipate?


## Let's assume we use 100 for validation, 300 for training
#### What problems do you anticipate?

## K-fold corss validation
 - Keras has no built in k-Fold corss validation function
 - Let's split the data ourselves

In [ ]:
def get_fold(k, i, X, y):
    """ Split into train and validataion data """

    return (Xf, Xv, yf, yv)

In [45]:
d = get_fold(4, 2, train_data, train_targets)
for x in d:
    print(x.shape)

(303, 13)
(101, 13)
(303,)
(101,)


In [44]:
# One possible solution
def get_fold(k, i, X, y):
    """ Split into train and validataion data """
    fold_len = len(y) // k
    ix = np.arange(len(y)) // fold_len == i
    nix = np.logical_not(ix)
    
    Xf = X[nix,:]
    Xv = X[ix, :]
    yf = y[nix]
    yv = y[ix]
    return (Xf, Xv, yf, yv)


(303, 13)
(101, 13)
(303,)
(101,)


In [ ]:
# Could also use
from sklearn.model_selection import KFold

In [19]:
# Used this way
for train, val in kFold.split(X, Y):
    model = create_model()
    model.fit(X[train], Y[train])
    model.evaluate(X[val], Y[val])

In [59]:
k = 4
num_val_samples = len(train_data) // k
all_scores = []
hists = []

for i in range(k):
    print('Processing fold #', i)
    Xf, Xv, yf, yv = get_fold(k, i, train_data, train_targets)
    model = create_model()
    # Train the model (in silent mode, verbose=0)
    history = model.fit(Xf, yf, epochs=30, batch_size=10,  verbose=0)
    # Evaluate the model on the validation data
    val_mse, val_mae = model.evaluate(Xv, yv, verbose=0)
    all_scores.append(val_mae)
    hists.append(history.history)

Processing fold # 0
Processing fold # 1
Processing fold # 2
Processing fold # 3


In [61]:
h = hists[0]
h.keys()

dict_keys(['loss', 'mean_absolute_error'])

## Scores

In [57]:
print(all_scores)

[2.4291117805065494, 2.8421869230742502, 2.5324113014901037, 2.5932457081162101]


In [52]:
np.mean(all_scores)

2.5992389282967783